# Capstone Project - The Battle of Neighborhoods

## Importing Libraries
In this section we are importing neccesery libraries.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

## Download and Explore Dataset
MORE HERE


In [2]:
#Downloading the dataset 
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
#Opening the dataset as json object
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
#newyork_data

In [5]:
#Accessing 'features' within the dataset
neighborhoods_data = newyork_data['features']

In [6]:
#neighborhoods_data[0]

In [7]:
#Creating new dataset, adding columnn and information to it from downloaded dataset
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough']
    neighborhood_name = data['properties']['name']
    
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
#Seeing the first 5 rows
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
#Looking at address
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

#Printing the amount of borughs and neighborhoods within the boroughs
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [10]:
#create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

#add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Accessing Foursquare venues

In [11]:
CLIENT_ID = 'JINXJXDKGLLTTSUIQM10IP4VPJIW52MKS0KSSD1MWP5QVEBY'
CLIENT_SECRET = 'IFRXJIQXI3A51RHS2ZDWB1DMUWBXHUOEBNGNXVT3OX2JG5FE' 
#Foursquare API version
VERSION = '20180605' 
LIMIT = 500
radius = 5000

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JINXJXDKGLLTTSUIQM10IP4VPJIW52MKS0KSSD1MWP5QVEBY
CLIENT_SECRET:IFRXJIQXI3A51RHS2ZDWB1DMUWBXHUOEBNGNXVT3OX2JG5FE


In [12]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            #create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            #make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            #return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [13]:
#Finding vegan venues on Manhattan
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)

newyork_venues_vegan = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1d3941735')

newyork_venues_vegan.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Kingsbridge-Riverdale Farmers' Market,40.879973,-73.907295,Vegetarian / Vegan Restaurant
1,Chinatown,40.715618,-73.994279,Jisu Vegetarian,40.716050,-73.995348,Vegetarian / Vegan Restaurant
2,Chinatown,40.715618,-73.994279,Petisco Vegano,40.714040,-73.988815,Vegetarian / Vegan Restaurant
3,Chinatown,40.715618,-73.994279,Orchard Grocer,40.717847,-73.990358,Vegetarian / Vegan Restaurant
4,Chinatown,40.715618,-73.994279,Beyond Sushi,40.722080,-73.996250,Vegetarian / Vegan Restaurant


In [14]:
newyork_venues_vegan.shape

(953, 7)

In [15]:
#Def to add markers to map
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [16]:
#Creating seperated map with markers
map_newyork_sushi = folium.Map(location=[latitude, longitude], zoom_start=10)

addToMap(newyork_venues_vegan, 'green', map_newyork)

map_newyork


In [17]:
#Def to add column to dataset
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try: 
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc [n, 'Venue']
        except: 
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [18]:
manhattan_grouped = newyork_venues_vegan.groupby('Neighborhood').count()

manhattan_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,14,14,14,14,14,14
Carnegie Hill,9,9,9,9,9,9
Central Harlem,6,6,6,6,6,6
Chelsea,38,38,38,38,38,38
Chinatown,42,42,42,42,42,42
Civic Center,35,35,35,35,35,35
Clinton,24,24,24,24,24,24
East Harlem,5,5,5,5,5,5
East Village,48,48,48,48,48,48


## Analyze the neighborhoods

In [19]:
#Creating a dummy dataset

manhattan_onehot = pd.get_dummies(newyork_venues_vegan[['Venue Category']], prefix="", prefix_sep="")

manhattan_onehot['Neighborhood'] = newyork_venues_vegan['Neighborhood']

fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])

manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Bagel Shop,Bakery,Burger Joint,Café,Coffee Shop,Deli / Bodega,Dessert Shop,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Food Truck,Gluten-free Restaurant,Grocery Store,Health Food Store,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Kosher Restaurant,Latin American Restaurant,Lounge,Mac & Cheese Joint,Market,Mediterranean Restaurant,Mexican Restaurant,New American Restaurant,Pizza Place,Restaurant,Salad Place,Sandwich Place,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [20]:
#Creating a grouped dataset
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

manhattan_grouped

,Neighborhood,American Restaurant,Asian Restaurant,Bagel Shop,Bakery,Burger Joint,Café,Coffee Shop,Deli / Bodega,Dessert Shop,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Food Truck,Gluten-free Restaurant,Grocery Store,Health Food Store,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Kosher Restaurant,Latin American Restaurant,Lounge,Mac & Cheese Joint,Market,Mediterranean Restaurant,Mexican Restaurant,New American Restaurant,Pizza Place,Restaurant,Salad Place,Sandwich Place,Sushi Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant
0,Battery Park City,0.000000,0.000000,0.000000,0.000000,0.071429,0.071429,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.428571
1,Carnegie Hill,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.777778
2,Central Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.833333
3,Chelsea,0.000000,0.026316,0.000000,0.026316,0.026316,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078947,0.026316,0.000000,0.026316,0.026316,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.0,0.000000,0.000000,0.026316,0.026316,0.026316,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.526316
4,Chinatown,0.000000,0.023810,0.000000,0.023810,0.000000,0.000000,0.000000,0.023810,0.023810,0.000000,0.000000,0.000000,0.023810,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.023810,0.047619,0.000000,0.023810,0.0,0.000000,0.000000,0.023810,0.023810,0.000000,0.047619,0.000000,0.023810,0.000000,0.000000,0.000000,0.642857
5,Civic Center,0.000000,0.028571,0.000000,0.000000,0.028571,0.028571,0.000000,0.000000,0.000000,0.000000,0.057143,0.000000,0.028571,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.057143,0.000000,0.000000,0.0,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.085714,0.000000,0.000000,0.000000,0.628571
6,Clinton,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.041667,0.041667,0.000000,0.000000,0.000000,0.583333
7,East Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
8,East Village,0.020833,0.000000,0.000000,0.000000,0.020833,0.000000,0.020833,0.020833,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.041667,0.000000,0.020833,0.0,0.020833,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.020833,0.000000,0.000000,0.020833,0.687500
9,Financial District,0.000000,0.000000,0.000000,0.000000,0.071429,0.071429,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.500000


In [21]:
#Printing 5 most recent venues in each neighborhood
num_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print('----'+hood+'----')
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_venues))
    print('\n')

----Battery Park City----
                           venue  freq
0  Vegetarian / Vegan Restaurant  0.43
1                    Salad Place  0.14
2             Falafel Restaurant  0.14
3                           Café  0.07
4                      Juice Bar  0.07


----Carnegie Hill----
                           venue  freq
0  Vegetarian / Vegan Restaurant  0.78
1                  Grocery Store  0.11
2                      Juice Bar  0.11
3               Sushi Restaurant  0.00
4                 Sandwich Place  0.00


----Central Harlem----
                           venue  freq
0  Vegetarian / Vegan Restaurant  0.83
1                         Market  0.17
2               Asian Restaurant  0.00
3                      Juice Bar  0.00
4              Kosher Restaurant  0.00


----Chelsea----
                           venue  freq
0  Vegetarian / Vegan Restaurant  0.53
1                     Food Truck  0.08
2                      Juice Bar  0.05
3                           Café  0.05
4         

In [22]:
#Def that returns the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
#Iterates through dataset and sorts venues from most common to 10th most common

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neighborhood_venues_sorted = pd.DataFrame(columns=columns)
neighborhood_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)
    
neighborhood_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Vegetarian / Vegan Restaurant,Salad Place,Falafel Restaurant,Juice Bar,Burger Joint,Café,Food Truck,Eastern European Restaurant,Grocery Store,Gluten-free Restaurant
1,Carnegie Hill,Vegetarian / Vegan Restaurant,Grocery Store,Juice Bar,Ice Cream Shop,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Dessert Shop
2,Central Harlem,Vegetarian / Vegan Restaurant,Market,Ice Cream Shop,Grocery Store,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Dessert Shop
3,Chelsea,Vegetarian / Vegan Restaurant,Food Truck,Salad Place,Juice Bar,Café,Bakery,Burger Joint,New American Restaurant,Mexican Restaurant,Mediterranean Restaurant
4,Chinatown,Vegetarian / Vegan Restaurant,Juice Bar,Pizza Place,Mediterranean Restaurant,Food Truck,Japanese Restaurant,Dessert Shop,Latin American Restaurant,Deli / Bodega,Grocery Store


## Clustering neighborhoods

In [31]:
#Start clusting neighborhoods together
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(manhattan_grouped_clustering)

kmeans.labels_[0:10]

array([0, 1, 1, 0, 3, 3, 3, 4, 3, 0], dtype=int32)

In [35]:
#neighborhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = neighborhoods
manhattan_merged = manhattan_merged.join(neighborhood_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [37]:
manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4.0,Vegetarian / Vegan Restaurant,Thai Restaurant,Health Food Store,Grocery Store,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Dessert Shop
1,Manhattan,Chinatown,40.715618,-73.994279,3.0,Vegetarian / Vegan Restaurant,Juice Bar,Pizza Place,Mediterranean Restaurant,Food Truck,Japanese Restaurant,Dessert Shop,Latin American Restaurant,Deli / Bodega,Grocery Store
2,Manhattan,Washington Heights,40.851903,-73.936900,2.0,Lounge,Vegetarian / Vegan Restaurant,Ice Cream Shop,Grocery Store,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Dessert Shop
3,Manhattan,Inwood,40.867684,-73.921210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4.0,Vegetarian / Vegan Restaurant,Thai Restaurant,Health Food Store,Grocery Store,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Dessert Shop


## Create a map

In [50]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]

markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

In [51]:

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Upper West Side,Vegetarian / Vegan Restaurant,Sandwich Place,Salad Place,Food Truck,Juice Bar,Health Food Store,Dessert Shop,Gluten-free Restaurant,Fast Food Restaurant,Falafel Restaurant
16,Murray Hill,Vegetarian / Vegan Restaurant,Salad Place,Juice Bar,New American Restaurant,Indian Restaurant,American Restaurant,Japanese Restaurant,Kosher Restaurant,Eastern European Restaurant,Fast Food Restaurant
17,Chelsea,Vegetarian / Vegan Restaurant,Food Truck,Salad Place,Juice Bar,Café,Bakery,Burger Joint,New American Restaurant,Mexican Restaurant,Mediterranean Restaurant
21,Tribeca,Vegetarian / Vegan Restaurant,Juice Bar,Salad Place,Falafel Restaurant,Café,Food Truck,Deli / Bodega,Mediterranean Restaurant,Burger Joint,Bakery
24,West Village,Vegetarian / Vegan Restaurant,Juice Bar,Salad Place,Asian Restaurant,Bakery,Burger Joint,Café,Deli / Bodega,Food Truck,Gluten-free Restaurant
28,Battery Park City,Vegetarian / Vegan Restaurant,Salad Place,Falafel Restaurant,Juice Bar,Burger Joint,Café,Food Truck,Eastern European Restaurant,Grocery Store,Gluten-free Restaurant
29,Financial District,Vegetarian / Vegan Restaurant,Falafel Restaurant,Salad Place,Juice Bar,Burger Joint,Café,Food Truck,Eastern European Restaurant,Grocery Store,Gluten-free Restaurant
33,Midtown South,Vegetarian / Vegan Restaurant,Juice Bar,Food Truck,Indian Restaurant,Salad Place,New American Restaurant,Japanese Restaurant,American Restaurant,Mexican Restaurant,Deli / Bodega
36,Tudor City,Vegetarian / Vegan Restaurant,Indian Restaurant,Salad Place,New American Restaurant,Sushi Restaurant,Food Truck,Japanese Restaurant,Juice Bar,Dessert Shop,Fast Food Restaurant
38,Flatiron,Vegetarian / Vegan Restaurant,Salad Place,Juice Bar,New American Restaurant,Indian Restaurant,Health Food Store,Sushi Restaurant,Asian Restaurant,Eastern European Restaurant,Food Truck


In [52]:

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Harlem,Vegetarian / Vegan Restaurant,Market,Ice Cream Shop,Grocery Store,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Dessert Shop
9,Yorkville,Vegetarian / Vegan Restaurant,Grocery Store,Juice Bar,Ice Cream Shop,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Dessert Shop
11,Roosevelt Island,Vegetarian / Vegan Restaurant,Salad Place,Dessert Shop,Grocery Store,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Deli / Bodega
13,Lincoln Square,Vegetarian / Vegan Restaurant,Italian Restaurant,Salad Place,Dessert Shop,Grocery Store,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant
20,Lower East Side,Vegetarian / Vegan Restaurant,Pizza Place,Mexican Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Dessert Shop,Mac & Cheese Joint,Deli / Bodega,Burger Joint
30,Carnegie Hill,Vegetarian / Vegan Restaurant,Grocery Store,Juice Bar,Ice Cream Shop,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Dessert Shop
31,Noho,Vegetarian / Vegan Restaurant,Juice Bar,Pizza Place,Food Truck,Salad Place,Bakery,Burger Joint,Mexican Restaurant,Grocery Store,Mac & Cheese Joint
34,Sutton Place,Vegetarian / Vegan Restaurant,Salad Place,Bagel Shop,New American Restaurant,Latin American Restaurant,Dessert Shop,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant
37,Stuyvesant Town,Vegetarian / Vegan Restaurant,Grocery Store,Mac & Cheese Joint,Ice Cream Shop,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Dessert Shop


In [53]:

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Lounge,Vegetarian / Vegan Restaurant,Ice Cream Shop,Grocery Store,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Dessert Shop


In [54]:

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Vegetarian / Vegan Restaurant,Juice Bar,Pizza Place,Mediterranean Restaurant,Food Truck,Japanese Restaurant,Dessert Shop,Latin American Restaurant,Deli / Bodega,Grocery Store
8,Upper East Side,Vegetarian / Vegan Restaurant,Grocery Store,Salad Place,Juice Bar,Falafel Restaurant,Dessert Shop,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Eastern European Restaurant
10,Lenox Hill,Vegetarian / Vegan Restaurant,Salad Place,Bagel Shop,Juice Bar,Latin American Restaurant,Falafel Restaurant,Eastern European Restaurant,Gluten-free Restaurant,Food Truck,Fast Food Restaurant
14,Clinton,Vegetarian / Vegan Restaurant,Juice Bar,American Restaurant,Deli / Bodega,Restaurant,Salad Place,Mexican Restaurant,Fast Food Restaurant,Sandwich Place,Food Truck
15,Midtown,Vegetarian / Vegan Restaurant,Juice Bar,Salad Place,New American Restaurant,Japanese Restaurant,American Restaurant,Indian Restaurant,Kosher Restaurant,Deli / Bodega,Eastern European Restaurant
18,Greenwich Village,Vegetarian / Vegan Restaurant,Salad Place,Juice Bar,Bakery,Pizza Place,Latin American Restaurant,Deli / Bodega,Food Truck,Gluten-free Restaurant,Grocery Store
19,East Village,Vegetarian / Vegan Restaurant,Pizza Place,Juice Bar,Mac & Cheese Joint,Burger Joint,Coffee Shop,Deli / Bodega,Dessert Shop,Thai Restaurant,Japanese Restaurant
22,Little Italy,Vegetarian / Vegan Restaurant,Juice Bar,Salad Place,Pizza Place,Grocery Store,Falafel Restaurant,Latin American Restaurant,Dessert Shop,Deli / Bodega,Coffee Shop
23,Soho,Vegetarian / Vegan Restaurant,Juice Bar,Salad Place,Bakery,Deli / Bodega,Pizza Place,Burger Joint,Café,Coffee Shop,Latin American Restaurant
25,Manhattan Valley,Vegetarian / Vegan Restaurant,Sandwich Place,Health Food Store,Asian Restaurant,Bagel Shop,Bakery,Burger Joint,Café,Coffee Shop,Ice Cream Shop


In [55]:

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Vegetarian / Vegan Restaurant,Thai Restaurant,Health Food Store,Grocery Store,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Dessert Shop
4,Hamilton Heights,Vegetarian / Vegan Restaurant,Thai Restaurant,Health Food Store,Grocery Store,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Dessert Shop
5,Manhattanville,Vegetarian / Vegan Restaurant,Thai Restaurant,Health Food Store,Grocery Store,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Dessert Shop
7,East Harlem,Vegetarian / Vegan Restaurant,Thai Restaurant,Health Food Store,Grocery Store,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Dessert Shop
26,Morningside Heights,Vegetarian / Vegan Restaurant,Thai Restaurant,Health Food Store,Grocery Store,Gluten-free Restaurant,Food Truck,Fast Food Restaurant,Falafel Restaurant,Eastern European Restaurant,Dessert Shop


## Results

Looking at the map I would open up a vegan restaurant on Broadway 72nd street to 116th street